In [ ]:
import json
import gc
import re
import unicodedata
from langdetect import detect_langs
import ollama


class NLPOrchestrator:
    def __init__(self, model_id="llama3.1:8b-instruct-q4_K_M"):
        print(f"Using Ollama model: {model_id}")
        self.model_id = model_id

        # Language mapping
        self.lang_map = {
            "en": "English",
            "hi": "Hindi",
            "te": "Telugu",
            "ta": "Tamil",
            "bn": "Bengali",
            "mr": "Marathi",
            "gu": "Gujarati",
            "kn": "Kannada",
            "ml": "Malayalam",
            "pa": "Punjabi",
            "ur": "Urdu",
            "or": "Odia",
            "as": "Assamese",
            "sa": "Sanskrit",
        }

    # -----------------------------
    # Core generation (deterministic)
    # -----------------------------
    def _generate(self, system_prompt, user_input, max_tokens=512):
        response = ollama.chat(
            model=self.model_id,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input},
            ],
            options={
                "temperature": 0.0,     # critical for JSON stability
                "num_predict": max_tokens,
            },
        )

        text = response["message"]["content"].strip()

        gc.collect()
        return text

    # -----------------------------
    # Text cleaning
    # -----------------------------
    def clean_text(self, text: str) -> str:
        text = unicodedata.normalize("NFKC", text)
        text = text.replace("\u200c", "").replace("\u200d", "")
        text = text.replace("\ufeff", "")
        text = re.sub(r"[ \t]+", " ", text)
        text = re.sub(r"\n{3,}", "\n\n", text)
        return text.strip()

    # -----------------------------
    # Romanized Hindi detector
    # -----------------------------
    def is_romanized_hindi(self, text: str) -> bool:
        if not text or len(text) < 20:
            return False

        ascii_ratio = sum(1 for c in text if ord(c) < 128) / max(len(text), 1)
        if ascii_ratio < 0.85:
            return False

        hinglish_markers = [
            "hai", "haan", "han", "nahi", "nahin", "kyu", "kyun", "kya", "kaise",
            "mera", "meri", "mere", "tum", "aap", "ap", "hum", "ham",
            "mat", "kar", "karo", "kariye", "krdo", "krna", "kr diya",
            "wala", "wali", "wale", "se", "ko", "me", "mein", "par",
            "bahut", "bohot", "thoda", "jaldi", "abhi", "kal", "aaj"
        ]

        text_l = " " + re.sub(r"[^a-zA-Z ]", " ", text.lower()) + " "
        hits = sum(1 for w in hinglish_markers if f" {w} " in text_l)

        return hits >= 2

    # -----------------------------
    # Language detection
    # -----------------------------
    def detect_language(self, text):
        try:
            if self.is_romanized_hindi(text):
                return {
                    "lang_list": ["Hindi (Romanized)"],
                    "primary_lang": "Hindi (Romanized)",
                }

            langs = detect_langs(text)
            primary = langs[0].lang
            lang_list = [self.lang_map.get(l.lang, l.lang.upper()) for l in langs[:3]]
            primary_lang = self.lang_map.get(primary, primary.upper())

            return {"lang_list": lang_list, "primary_lang": primary_lang}

        except Exception:
            return {"lang_list": ["Unknown"], "primary_lang": "Unknown"}

    # -----------------------------
    # JSON parsing helpers
    # -----------------------------
    def _try_load_json(self, json_str: str):
        if not json_str:
            return None

        json_str = json_str.strip()
        json_str = re.sub(r",\s*}", "}", json_str)
        json_str = re.sub(r",\s*]", "]", json_str)
        json_str = json_str.replace("“", "\"").replace("”", "\"").replace("’", "'")

        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None

    def _parse_json(self, response: str):
        if not response:
            return None

        match = re.search(r"```json\s*([\s\S]*?)\s*```", response, re.IGNORECASE)
        if match:
            parsed = self._try_load_json(match.group(1))
            if parsed:
                return parsed

        match = re.search(r"```\s*([\s\S]*?)\s*```", response)
        if match:
            parsed = self._try_load_json(match.group(1))
            if parsed:
                return parsed

        match = re.search(r"\{[\s\S]*\}", response)
        if match:
            parsed = self._try_load_json(match.group(0))
            if parsed:
                return parsed

        print("⚠️ JSON parsing failed")
        return None

    # -----------------------------
    # Step 1: Translation
    # -----------------------------
    def step_1_translate(self, text, lang_info):
        primary = lang_info["primary_lang"].lower()

        if primary == "english":
            return {"translated_english_text": text, "translation_confidence": 1.0}

        system_prompt = """You are a professional translator.
Translate the given text into fluent English.

Rules:
- Translate completely
- Preserve meaning and proper nouns
- Return ONLY valid JSON

Output JSON format:
{
  "translated_english_text": "...",
  "translation_confidence": 0.0
}
"""

        response = self._generate(system_prompt, text, max_tokens=800)
        result = self._parse_json(response)

        if result is None:
            return {"translated_english_text": response, "translation_confidence": 0.6}

        return result

    # -----------------------------
    # Step 2: Deep analysis
    # -----------------------------
    def step_2_deep_analysis(self, english_text):
        system_prompt = """You are a security-focused NLP analyzer.
Perform comprehensive analysis on the given text.

CRITICAL RULES:
1. Country_iden: Select ONLY ONE value based on content context
2. Domains: Use exact capitalization from the list
3. Location NER: Only specific geographic places
4. Event dates: dd/mm/yyyy ONLY if explicitly mentioned
5. Sentiment "Anti-National": Only for direct threats to India

DOMAIN OPTIONS:
Politics, Crime, Military, Terrorism, Radicalisation, Extremism in J&K,
Law and Order, Narcotics, Left Wing Extremism, General

Return ONLY valid JSON in this format:
{
  "domain_ident": [],
  "sentiment": "",
  "NER": {
    "Person": [],
    "Location": [],
    "Organisation": [],
    "Event": [],
    "Product": []
  },
  "Event_calendar": [],
  "Country_iden": "",
  "Fact_checker": {
    "relevant_topics": [],
    "confidence_level": 0.0,
    "relevance_rating": ""
  },
  "Summary": ""
}
"""

        response = self._generate(system_prompt, english_text, max_tokens=1000)
        result = self._parse_json(response)

        if result is None:
            return {
                "domain_ident": ["General"],
                "sentiment": "Neutral",
                "NER": {
                    "Person": [],
                    "Location": [],
                    "Organisation": [],
                    "Event": [],
                    "Product": [],
                },
                "Event_calendar": [],
                "Country_iden": "Abroad",
                "Fact_checker": {
                    "relevant_topics": [],
                    "confidence_level": 0.0,
                    "relevance_rating": "Low",
                },
                "Summary": "Analysis failed.",
            }

        return result

    # -----------------------------
    # Orchestration
    # -----------------------------
    def process(self, text):
    cleaned = self.clean_text(text)

    lang_info = self.detect_language(cleaned)
    translation = self.step_1_translate(cleaned, lang_info)
    analysis = self.step_2_deep_analysis(
        translation["translated_english_text"]
    )

    return {
        "Cleaned_content": cleaned,
        "lang_list": lang_info["lang_list"],
        "primary_lang": lang_info["primary_lang"],
        "translated_english_text": translation["translated_english_text"],
        **analysis,
    }

    def process_batch(self, texts):
    """
    texts: List[str]
    returns: List[Dict]
    """
    results = []

    for text in texts:
        try:
            output = self.process(text)
        except Exception as e:
            output = {
                "error": str(e),
                "raw_input": text
            }
        results.append(output)

    return results




In [3]:
content='''கடந்த பதினைந்து நாட்களாக உலகில் என்ன நடந்தது?

இந்த மாத தொடக்கத்தில் வெனிசுவேலாவில் மேற்கொள்ளப்பட்ட வெற்றிகரமான ராணுவ நடவடிக்கையால் உற்சாகமடைந்த டொனால்ட் டிரம்ப் , கிரீன்லாந்து விவகாரத்தில் தனது ஆக்ரோஷமான பேச்சைத் தொடங்கினார்.

கிரீன்லாந்து மீதான உரிமை கோரல்கள், ராணுவ நடவடிக்கை குறித்த எச்சரிக்கைகள் மற்றும் ஐரோப்பாவின் பாரம்பரிய நட்பு நாடுகளுக்கு எதிரான வர்த்தக வரிகள் என உலகம் தினமும் ஒரு செய்தியை எதிர்கொண்டது.

ஆனால் இப்போது, இவை அனைத்தும் ஒரு புகையைப் போல மறைந்துவிட்டதாகத் தெரிகிறது.

டிரம்பை கையாள்வதில் வல்லவர் என்று கருதப்படும் நேட்டோ பொதுச் செயலாளர் மார்க் ருட்டே, அதிபரின் இந்த ஆபத்தான போக்கைக் கட்டுப்படுத்தியதாகத் தெரிகிறது.

கடந்த வாரம் டென்மார்க் மற்றும் கிரீன்லாந்து வெளியுறவு அமைச்சர்கள் அமெரிக்காவுக்கு மேற்கொண்ட பயணத்தின் போது இதற்கான அடித்தளம் அமைக்கப்பட்டிருக்கலாம்.

அந்தப் பயணத்தின் முடிவில், கிரீன்லாந்தின் எதிர்காலம் குறித்து விவாதிக்க ஒரு "செயற்குழு" அமைக்க ஒப்புக்கொள்ளப்பட்டது.

வடக்கு அட்லாண்டிக் கூட்டணியையே சிதைக்கக்கூடிய ஒரு சிக்கலை ரூட்டே மிக நுட்பமாக கையாண்டுள்ளதாகத் தெரிகிறது.'''

In [ ]:
if __name__ == "__main__":
    orchestrator = NLPOrchestrator()

    test_inputs = [
        "Mujhe aaj bhi yaad hai woh din, 2 April 2011...",
        "India conducted a military exercise near the border.",
        "The suspect was arrested by Delhi Police on Monday."
    ]

    results = orchestrator.process_batch(test_inputs)

    print(json.dumps(results, ensure_ascii=False))


Using Ollama model: llama3.1:8b-instruct-q4_K_M
NLP ORCHESTRATION PIPELINE
🔍 Detecting language...
📝 Translation...
⚠️ JSON parsing failed
🔬 Deep analysis...

FINAL OUTPUT
{
  "Cleaned_content": "கடந்த பதினைந்து நாட்களாக உலகில் என்ன நடந்தது?\n\nஇந்த மாத தொடக்கத்தில் வெனிசுவேலாவில் மேற்கொள்ளப்பட்ட வெற்றிகரமான ராணுவ நடவடிக்கையால் உற்சாகமடைந்த டொனால்ட் டிரம்ப் , கிரீன்லாந்து விவகாரத்தில் தனது ஆக்ரோஷமான பேச்சைத் தொடங்கினார்.\n\nகிரீன்லாந்து மீதான உரிமை கோரல்கள், ராணுவ நடவடிக்கை குறித்த எச்சரிக்கைகள் மற்றும் ஐரோப்பாவின் பாரம்பரிய நட்பு நாடுகளுக்கு எதிரான வர்த்தக வரிகள் என உலகம் தினமும் ஒரு செய்தியை எதிர்கொண்டது.\n\nஆனால் இப்போது, இவை அனைத்தும் ஒரு புகையைப் போல மறைந்துவிட்டதாகத் தெரிகிறது.\n\nடிரம்பை கையாள்வதில் வல்லவர் என்று கருதப்படும் நேட்டோ பொதுச் செயலாளர் மார்க் ருட்டே, அதிபரின் இந்த ஆபத்தான போக்கைக் கட்டுப்படுத்தியதாகத் தெரிகிறது.\n\nகடந்த வாரம் டென்மார்க் மற்றும் கிரீன்லாந்து வெளியுறவு அமைச்சர்கள் அமெரிக்காவுக்கு மேற்கொண்ட பயணத்தின் போது இதற்கான அடித்தளம் அமைக்கப்பட்டிருக்கலாம்.\n\nஅந